##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# LoggingTensorHook 및 StopAtStepHook을 Keras 콜백으로 마이그레이션

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/logging_stop_hook"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/logging_stop_hook.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/logging_stop_hook.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/logging_stop_hook.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

TensorFlow 1에서는 사용 `tf.estimator.LoggingTensorHook` 하면서, 텐서를 모니터링하고 기록하는 `tf.estimator.StopAtStepHook` 지정된 단계에서 정지 훈련을하는 데 도움이 때와 훈련 `tf.estimator.Estimator` . 이 노트북은 사용자 정의 Keras 콜백 (사용 TensorFlow 2에서 그 등가물에 이러한 API에서 마이그레이션하는 방법을 보여줍니다 `tf.keras.callbacks.Callback` 포함) `Model.fit` .

Keras [콜백](https://www.tensorflow.org/guide/keras/custom_callback) `Model.fit` / `Model.evaluate` / `Model.predict` API에서 학습/평가/예측 중에 서로 다른 지점에서 호출되는 객체입니다. 콜백에 대한 자세한 내용은 `tf.keras.callbacks.Callback` API 문서와 [자체 콜백 작성](../..guide/keras/custom_callback.ipynb/) [및 내장 메서드를 사용한 교육 및 평가](https://www.tensorflow.org/guide/keras/train_and_evaluate) ( *콜백 사용* 섹션) 가이드를 참조하세요. `SessionRunHook` 에서 TensorFlow 2의 Keras 콜백으로 [마이그레이션하려면 지원 논리를 사용한 마이그레이션 교육](sessionrunhook_callback.ipynb) 가이드를 확인하세요.

## 설정

데모용으로 가져오기 및 간단한 데이터세트로 시작합니다.

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]

# Define an input function.
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

## TensorFlow 1: tf.estimator API를 사용하여 텐서를 기록하고 학습을 중지합니다.

TensorFlow 1에서는 훈련 동작을 제어하기 위해 다양한 후크를 정의합니다. 그런 다음 이 후크를 `tf.estimator.EstimatorSpec` 전달합니다.

아래 예에서:

- 텐서(예: 모델 가중치 또는 손실)를 모니터링/로그하려면 `tf.estimator.LoggingTensorHook` ( `tf.train.LoggingTensorHook` 은 별칭)을 사용합니다.
- 특정 단계에서 훈련을 중지하려면 `tf.estimator.StopAtStepHook` ( `tf.train.StopAtStepHook` 은 별칭)을 사용합니다.

In [ ]:
def _model_fn(features, labels, mode):
  dense = tf1.layers.Dense(1)
  logits = dense(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())

  # Define the stop hook.
  stop_hook = tf1.train.StopAtStepHook(num_steps=2)

  # Access tensors to be logged by names.
  kernel_name = tf.identity(dense.weights[0])
  bias_name = tf.identity(dense.weights[1])
  logging_weight_hook = tf1.train.LoggingTensorHook(
      tensors=[kernel_name, bias_name],
      every_n_iter=1)
  # Log the training loss by the tensor object.
  logging_loss_hook = tf1.train.LoggingTensorHook(
      {'loss from LoggingTensorHook': loss},
      every_n_secs=3)

  # Pass all hooks to `EstimatorSpec`.
  return tf1.estimator.EstimatorSpec(mode,
                                     loss=loss,
                                     train_op=train_op,
                                     training_hooks=[stop_hook,
                                                     logging_weight_hook,
                                                     logging_loss_hook])

estimator = tf1.estimator.Estimator(model_fn=_model_fn)

# Begin training.
# The training will stop after 2 steps, and the weights/loss will also be logged.
estimator.train(_input_fn)

## TensorFlow 2: 사용자 지정 콜백 및 Model.fit을 사용하여 텐서를 기록하고 훈련을 중지합니다.

TensorFlow 2에서 `Model.fit` (또는 `Model.evaluate` `tf.keras.callbacks.Callback` 을 정의하여 텐서 모니터링 및 학습 중지를 구성할 수 있습니다. 그런 다음 이를 `Model.fit` (또는 `Model.evaluate` ) `callbacks` 매개변수에 전달합니다. [(자신만의 콜백 작성](../..guide/keras/custom_callback.ipynb) 가이드에서 자세히 알아보세요.)

아래 예에서:

- `StopAtStepHook` 의 기능을 다시 생성하려면 특정 단계 수 후에 훈련을 중지 `on_batch_end` 메서드를 재정의하는 사용자 지정 콜백(아래에서 `StopAtStepCallback`
- `LoggingTensorHook` 동작을 다시 생성하려면 이름으로 텐서에 액세스하는 것이 지원되지 않으므로 로깅된 텐서를 수동으로 기록하고 출력하는 사용자 지정 콜백( `LoggingTensorCallback` 사용자 정의 콜백 내에서 로깅 빈도를 구현할 수도 있습니다. 아래 예에서는 두 단계마다 가중치를 인쇄합니다. N초마다 기록하는 것과 같은 다른 전략도 가능합니다.

In [ ]:
class StopAtStepCallback(tf.keras.callbacks.Callback):
  def __init__(self, stop_step=None):
    super().__init__()
    self._stop_step = stop_step

  def on_batch_end(self, batch, logs=None):
    if self.model.optimizer.iterations >= self._stop_step:
      self.model.stop_training = True
      print('\nstop training now')

class LoggingTensorCallback(tf.keras.callbacks.Callback):
  def __init__(self, every_n_iter):
      super().__init__()
      self._every_n_iter = every_n_iter
      self._log_count = every_n_iter

  def on_batch_end(self, batch, logs=None):
    if self._log_count > 0:
      self._log_count -= 1
      print("Logging Tensor Callback: dense/kernel:",
            model.layers[0].weights[0])
      print("Logging Tensor Callback: dense/bias:",
            model.layers[0].weights[1])
      print("Logging Tensor Callback loss:", logs["loss"])
    else:
      self._log_count -= self._every_n_iter

완료되면 새로운 콜백인 `StopAtStepCallback` 및 `LoggingTensorCallback` 을 Model.fit의 `callbacks` 매개변수에 `Model.fit` .

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)
model.compile(optimizer, "mse")

# Begin training.
# The training will stop after 2 steps, and the weights/loss will also be logged.
model.fit(dataset, callbacks=[StopAtStepCallback(stop_step=2),
                              LoggingTensorCallback(every_n_iter=2)])

## 다음 단계

콜백에 대해 자세히 알아보기:

- API 문서: `tf.keras.callbacks.Callback`
- 가이드: [자신만의 콜백 작성하기](../..guide/keras/custom_callback.ipynb/)
- 가이드: [기본 제공 메서드를 사용한 교육 및 평가](https://www.tensorflow.org/guide/keras/train_and_evaluate) ( *콜백 사용* 섹션)

다음과 같은 마이그레이션 관련 리소스도 유용할 수 있습니다.

- [조기 중지 마이그레이션 가이드](early_stopping.ipynb) : `tf.keras.callbacks.EarlyStopping` 은 조기 중지 콜백이 내장되어 있습니다.
- [TensorBoard 마이그레이션 가이드](tensorboard.ipynb) : TensorBoard는 측정항목을 추적하고 표시할 수 있습니다.
- [지원 로직 마이그레이션 가이드를 통한 교육](sessionrunhook_callback.ipynb) : `SessionRunHook` 에서 TensorFlow 2의 Keras 콜백까지